In [ ]:
import numpy as np
import csv

In [ ]:
RND_MEAN = 0
RND_STD = 0.0030
LEARNING_RATE = 0.0001

In [ ]:
def steel_exec(epoch_count=10, mb_size=10, report=1):
  load_steel_dataset()
  init_model()
  train_and_test(epoch_count, mb_size, report)

In [ ]:
def load_steel_dataset():
  with open("/content/drive/MyDrive/ml-data/Faults/faults.csv") as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None)
    rows = []
    for row in csvreader:
      rows.append(row)
  
  global data, input_cnt, output_cnt
  input_cnt, output_cnt = 27, 7
  data = np.asarray(rows, dtype="float32")

In [ ]:
def init_model():
  global weight, bias, input_cnt, output_cnt
  weight = np.random.normal(RND_MEAN, RND_STD, (input_cnt, output_cnt))
  bias = np.zeros(output_cnt)

In [ ]:
def train_and_test(epoch_count, mb_size, report):
  step_count = arrange_data(mb_size)
  test_x, test_y = get_test_data()
  for epoch in range(epoch_count):
    losses = []
    for n in range(step_count):
      train_x, train_y = get_train_data(mb_size, n)
      loss, _ = run_train(train_x, train_y)
      losses.append(loss)
    if report > 0 and (epoch + 1) % report == 0:
      acc = run_test(test_x, test_y)
      print("Epoch {} | loss:{:5.3f}, accuracy={:5.3f}".\
            format(epoch+1, np.mean(losses), acc))
  final_acc = run_test(test_x, test_y)
  print("FINAL TEST result={:5.3f}".format(final_acc))

In [ ]:
def run_train(x, y):
  output, forward_info = forward_neuralnet(x)
  loss, post_info = forward_postproc(output, y)
  accuracy = eval_accuracy(output, y)

  G_loss = 1.
  G_output = backprop_postproc(G_loss, post_info)
  backprop_neuralnet(G_output, forward_info)
  return loss, accuracy

In [ ]:
def forward_postproc(output, y):
  entropy = softmax_cross_entropy_with_logits(y, output)
  loss = np.mean(entropy)
  return loss, [y, output, entropy]

def backprop_postproc(G_loss, post_info):
  y, output, entropy = post_info
  g_loss_entropy = 1.0 / np.prod(entropy.shape)
  g_entropy_output = softmax_cross_entropy_with_logits_derv(y, output)

  print("########################")
  print(entropy.shape)
  print(g_loss_entropy)
  G_entropy = g_loss_entropy * G_loss
  G_output = g_entropy_output * G_entropy
  print(G_output.shape)
  return G_output

In [ ]:
def softmax(x):
  max_elements = np.max(x, axis=1)
  diff = (x.T - max_elements).T
  exp = np.exp(diff)
  sum_exp = np.sum(exp, axis=1)
  probs = (exp.T / sum_exp).T
  return probs

def softmax_derv(x, y):
  mb_size, nom_size = x.shape
  derv = np.zeros((mb_size, nom_size, nom_size))
  for n in range(mb_size):
    for i in range(nom_size):
      for j in range(nom_size):
        derv[n, i, j] = - y[n,i] * y[n,j]
      derv[n, i, i] += y[n, i]
  return derv

def softmax_cross_entropy_with_logits(labels, logits):
  probs = softmax(logits)
  return -np.sum(labels * np.log(probs + 1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
  return softmax(logits) - labels  


In [ ]:
def forward_neuralnet(x):
  output = np.matmul(x, weight) + bias
  return output, x

def backprop_neuralnet(G_output, x):
  global weight, bias
  G_w = np.matmul(x.T, G_output)
  G_b = np.sum(G_output, axis=0)

  weight -= LEARNING_RATE * G_w
  bias -= LEARNING_RATE * G_b

In [ ]:
def get_test_data():
  test_index = shuffle_map[test_begin_index:]
  test_data = data[test_index]
  return test_data[:,:-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, n):
  if n == 0:
    np.random.shuffle(shuffle_map)
  train_index = shuffle_map[mb_size*n:mb_size*(n+1)]
  train_data = data[train_index]
  return train_data[:,:-output_cnt], train_data[:,-output_cnt:]

In [ ]:
def arrange_data(mb_size):
  global data, shuffle_map, test_begin_index
  shuffle_map = np.arange(data.shape[0])
  np.random.shuffle(shuffle_map)
  step_count = int(data.shape[0] * 0.8) // mb_size
  test_begin_index = step_count * mb_size
  return step_count

In [ ]:
def eval_accuracy(y, output):
  y_ = np.argmax(y, axis=1)
  output_ = np.argmax(output, axis=1)
  correct = np.equal(y_, output_)
  return np.mean(correct)

In [ ]:
def run_test(x, y):
  output, _ = forward_neuralnet(x)
  accuracy = eval_accuracy(y, output)
  return accuracy

In [ ]:
steel_exec(epoch_count=1000, mb_size=10, report=10)

########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
########################
(10,)
0.1
(10, 7)
###########

KeyboardInterrupt: ignored